In [30]:
import pandas as pd
import os
import openpyxl
import xlrd
import regex as re
from openpyxl import workbook

In [31]:
#global variables
bat_list,bowl_list,stad_list=[],[],[]
data,batsman_data,bowler_data,stad_data="","","",""
batsmen,bowlers,stadiums=dict(),dict(),dict()
bowling_order=[]

In [32]:
#data = pd.read_csv("test_data.csv")
batsman_data=pd.read_csv("batsman_data.csv",header=0)
bowler_data=pd.read_csv("bowler_data.csv",header=0)
#stad_data=pd.read_csv("stadium_data.csv")

In [33]:
wb_sheet=openpyxl.load_workbook('test_data.xlsx')
ws_sheet=wb_sheet.active

In [34]:
for row in range(0,batsman_data.index[-1]):
    player_name1 = batsman_data.at[row,'Player_Name']
    player_name = player_name1[0:1] +" "+ player_name1[len(player_name1)-4 : len(player_name1)+1]
    batsmen.update({player_name:row})
for row in range(0,bowler_data.index[-1]):
    player_name1 = bowler_data.at[row,'Player_Name']
    player_name = player_name1[0:1] +" "+ player_name1[len(player_name1)-4 : len(player_name1)+1]
    bowlers.update({player_name:row})
    
#for row in range(0,stad_data.index[-1]):
#    player_name1 = stad_data.at[row,'stadium_name']
#    player_name = player_name1[0:1] +" "+ player_name1[len(player_name1)-4 : len(player_name1)+1]
#    stadiums.update({player_name:row})

In [35]:
def get_batsman_value(b,col):
    b=b.strip()
    player_name = b[0:1] +" "+ b[len(b)-4 : len(b)+1]
    row= batsmen.get(player_name)
    try:
        data=batsman_data.at[row,col]
    except KeyError:
        data=0
    return data

def get_bowler_value(b,col):
    b=b.strip()
    player_name = b[0:1] +" "+ b[len(b)-4 : len(b)+1]

    row= bowlers.get(player_name)
    try:
        data= bowler_data.at[row,col]
    except KeyError:
        data=0
    return data

#def get_stadium_value(b,col):
#    b=b.strip()
#    player_name = b[0:1] +" "+ b[len(b)-4 : len(b)+1]
#    row= stadiums.get(player_name)
#    return stad_data.at[row,col]

In [36]:
#functions for batting calculations
def p_bat(b,k):
    col="p"+str(k+1)
    prob = get_batsman_value(b,col)
    if prob==0 or prob==None:
        col='wicket_probability'
        prob = get_batsman_value(b,col)
    if prob==0 or prob==None:
        prob=batsman_data.at[0,'average_wicket_probability']
    return prob

def r_bat(b,n):
    col="r"+str(n+1)
    score =get_batsman_value(b,col)
    if score==0 or score == None:
        col='strike_rate'
        score=get_batsman_value(b,col)
    if score==0:
        score=batsman_data.at[0,'average_strike_rate']
    return score

def s_bat(bx,by,o):
    if o<6 and bx in bat_list and by in bat_list:
        #figure out next and later batsmen
        idx= max(bat_list.index(bx),bat_list.index(by))
        if idx+1<len(bat_list):
            nxt=bat_list[idx+1]
        else:
            nxt="gen"
            later="gen"
        if idx+2<len(bat_list):
            later=bat_list[idx+2]
        else:
            later="gen"
            
        a=0.5
        
        px= s_bat(by,nxt,o+1)+a*r_bat(bx,o)+r_bat(by,o)
        py= s_bat(bx,nxt,o+1)+r_bat(bx,o)+a*r_bat(by,o)
        pxy=s_bat(nxt,later,o+1)+ a*(r_bat(bx,o)+r_bat(by,o))
        p0= s_bat(bx,by,o+1)+r_bat(bx,o)+r_bat(by,o)
        #if none can get out, we will not find any more probabilities
        if nxt=="gen":
            return p0
        #if one more can get out then we calculate the probability accordingly
        elif later=="gen":
            return p_bat(bx,o)*px + p_bat(by,o)*py + (1-p_bat(bx,o)-p_bat(by,o))*p0
        #if more than one can get out then also we calculate the probability accordingly
        else:
            return p_bat(bx,o)*px + p_bat(by,o)*py + p_bat(bx,o)*p_bat(by,o)*pxy + (1-p_bat(bx,o)-p_bat(by,o)- p_bat(bx,o)*p_bat(by,o))*p0
    else:
        return 0

In [37]:
def bowling_prediction(bowling_order):
    score=0
    wickets=0
    for player in bowling_order:
        #get score
        s=get_bowler_value(player,'pp_economy')
        if s==0:
            s=bowler_data.at[0,'avg_pp_econ']
        score+=s
        #get wickets
        w=get_bowler_value(player,'wicket_economy')
        if w==0:
            w=bowler_data.at[0,'avg_pp_wicket']
        wickets+=w
    return score

In [38]:
def get_bowling_order(bowl_list):
    # create a list of bowlers bowl[0-5] from bowl_list[] based on bowling pattern of the team
    bowling_order=[]
    if len(bowl_list) == 2:
        for i in range(0, 6):
            bowling_order.append(bowl_list[i % 2])
    # for bowler length 3 and 4
    # add team dependent bowling strategy
    elif len(bowl_list) == 3:
        bowling_order = (bowl_list[0], bowl_list[1], bowl_list[0], bowl_list[1], bowl_list[0], bowl_list[2])
    elif len(bowl_list) == 4:
        bowling_order = (bowl_list[0], bowl_list[1], bowl_list[0], bowl_list[1], bowl_list[2], bowl_list[3])
    elif len(bowl_list) == 5:
        bowling_order = (bowl_list[0], bowl_list[1], bowl_list[0], bowl_list[2], bowl_list[3], bowl_list[4])
    elif len(bowl_list) == 6:
        bowling_order = bowl_list
    return bowling_order

In [41]:
#transfer data into sheet of input output
stadium_data=pd.read_csv("stadium_data.csv")
stadiums=dict()

for row in range(0,stadium_data.index[-1]):
    stadium = stadium_data.at[row,'stadium_name']
    stadiums.update({stadium:row})

for rowx in range(2,ws_sheet.max_row+1):
    venue=ws_sheet.cell(row=rowx,column=1).value
    innings=ws_sheet.cell(row=rowx,column=2).value
    batting_team=ws_sheet.cell(row=rowx,column=3).value
    bowling_team=ws_sheet.cell(row=rowx,column=4).value
    bat_list=list(map(str,ws_sheet.cell(row=rowx,column=5).value.split(",")))
    bowl_list=list(map(str,ws_sheet.cell(row=rowx,column=6).value.split(",")))
    
    for i in bat_list:
        if i == " ":
            bat_list.remove(i)
    for i in bowl_list:
        if i==" ":
            bowl_list.remove(i)
    bowling_order=get_bowling_order(bowl_list)
    #row is the stadium row and column is
    batsman_pred=s_bat(bat_list[0],bat_list[1],0)
    bowler_pred=bowling_prediction(bowling_order)
    wickets=len(bat_list)-2
    prediction = 0.37091807*bowler_pred - 3.83388431*wickets + 33.87871819102332
    stadium_runs= stadium_data.at[stadiums.get(venue),'avg_score'+str(innings)]
    stadium_wickets=stadium_data.at[stadiums.get(venue),'avg_wickets'+str(innings)]
    
    ws_sheet.cell(row=rowx,column=8,value=batsman_pred)
    ws_sheet.cell(row=rowx,column=9,value=bowler_pred)
    ws_sheet.cell(row=rowx,column=10,value=stadium_runs)
    ws_sheet.cell(row=rowx,column=11,value=stadium_wickets)
    ws_sheet.cell(row=rowx,column=12,value=prediction)


In [42]:
#transfer data into sheet of input output
wb_sheet.save("test_data.xlsx")

In [ ]:
#Using the idea of the team that is bowling, we generate a pattern of bowlers that bowl in overs 1-6
#CSK	3->121213
#	4->121234
#DC	3->121313
#	4->121234
#KKR	3->121323
#	4->121234
#MI	3->121312
#	4->121234
#PK	3->121323
#	4->121343
#RR	3->121323
#	4->1213xx
#RCB	3->121213
#	4->123414
#SRH	3->121231
#	4->121234
#	
#5gen->121345	
#6->123456	


In [23]:
def get_data(row):
    venue=data.iloc[row,0]
    innings=data.iloc[row,1]
    batting_team=data.iloc[row,2]
    bowling_team=data.iloc[row,3]
    bat_list=list(map(str,data.iloc[row,4].split(",")))
    bowl_list=list(map(str,data.iloc[row,5].split(",")))
    for i in bat_list:
        if i == " ":
            bat_list.remove(i)
    for i in bowl_list:
        if i==" ":
            bowl_list.remove(i)
    print(bat_list)

In [ ]:
#unused codes

In [8]:
#functions for bowling calculations
def p_bowl(b,n):
    col="p"+str(n+1)
    prob=get_bowler_value(b,col)    
    if prob==0 or prob==None:
        col='wicket_rate'
        prob=get_bowler_value(b,col)
    if prob==0:
        prob=bowler_data.at[0,'average_wicket_rate']
    if n==1:
        return prob
    if n==2:
        return prob*prob*5/6
    if n==0:
        return (1-prob*(1+prob*5/6))

def c_bowl(b,n,w):
    col="r("+str(n+1)+str(w)+")"
    val = get_bowler_value(b,col)
    #if val is 0, we need to return the general economy of the bowler
    if val==0 or val==None:
        col='economy'
        val = get_bowler_value(b,col)
    if val==0:
        val = bowler_data.at[0,'average_economy']
    return val

def t_bowl(n,w,total):#runs conceded starting from 0th over to 5th over (0-5) range
    #total is the total number of batsmen
    if n<6 and w<=total-2:
        
        t0=t_bowl(n+1,w,total)
        t1=t_bowl(n+1,w+1,total)
        t2=t_bowl(n+1,w+2,total)
        
        p0= p_bowl(bowling_order[n],0)
        p1= p_bowl(bowling_order[n],1)
        p2= p_bowl(bowling_order[n],2)
        
        c= c_bowl(bowling_order[n],n,w)
        
        if total-w>=4:
            score= c + p0*t0 + p1*t1 + p2*t2
        elif total-w==3:
            score= c + (p0+p2)*t0 + p1*t1
        else:
            score= c + t0
        return score
    else: return 0

In [13]:
#transfer data into sheet of input output
input_file=pd.read_csv("inputFile.csv")
stadium_data=pd.read_csv("stadium_data.csv")
stadiums=dict()

for row in range(0,stadium_data.index[-1]):
    stadium = stadium_data.at[row,'stadium_name']
    stadiums.update({stadium:row})

for rowx in range(2,ws_sheet.max_row-1):
    venue=ws_sheet.cell(row=rowx,column=1).value
    innings=ws_sheet.cell(row=rowx,column=2).value
    batting_team=ws_sheet.cell(row=rowx,column=3).value
    bowling_team=ws_sheet.cell(row=rowx,column=4).value
    bat_list=list(map(str,ws_sheet.cell(row=rowx,column=5).value.split(",")))
    bowl_list=list(map(str,ws_sheet.cell(row=rowx,column=6).value.split(",")))
    
    for i in bat_list:
        if i == " ":
            bat_list.remove(i)
    for i in bowl_list:
        if i==" ":
            bowl_list.remove(i)
    #row is the stadium row and column is 
    stadium_runs= stadium_data.at[stadiums.get(venue),'avg_score'+str(innings)]
    stadium_wickets=stadium_data.at[stadiums.get(venue),'avg_wickets'+str(innings)]
    ws_sheet.cell(row=rowx,column=16,value=stadium_runs)
    ws_sheet.cell(row=rowx,column=17,value=stadium_wickets)

In [2]:
import pandas as pd

dict1 = {"number of storage arrays": 45, "number of ports":2390}

df = pd.DataFrame(data=dict1, index=[0])

df = (df.T)

print (df)


                             0
number of storage arrays    45
number of ports           2390
